## Bounce-averaged advection and diffusion coefficients for monochromatic electromagnetic ion cyclotron wave: Comparison between test-particle and quasi-linear models

Su, Zhu, Xiao, Zheng, Shen, Wang, Wang  (2012)


In [5]:
import numpy as np
import os
import sys
import inspect
import matplotlib.pyplot as plt


current_dir =  os.path.abspath(os.path.dirname('__file__'))

fpath_envi = os.path.abspath(current_dir + "../../../")
fpath_wave = os.path.abspath(current_dir + "../../../")
fpath_wpi = os.path.abspath(current_dir + "../../../wave_particle_interactions/parallel_EMIC_mod")

sys.path.append(fpath_envi)
sys.path.append(fpath_wave)
sys.path.append(fpath_wpi)

print(fpath_wave)
import environment_mod as env
import waveproperties_mod as wave

/home/tourgaidis/Επιφάνεια εργασίας/WPIT_GIT/WPIT


### 1. Define simulation parameters

Here we define all the initial parameters of the simulation in respect with the particle and the wave

In [ ]:
from environment_mod import Bmag_dipole
from environment_mod import aeq2alpha
from environment_mod import cyclotron
from environment_mod import densities_denton,omega_plasma
from environment_mod import aeq2alpha
from waveproperties_mod import ref_index_parallel_EMIC
from environment_mod import momentums
from environment_mod import dwc_ds
from parallel_EMIC_mod import nonlinear_S,nonlinear_H,nonlinear_theta

In [ ]:
L_sim=4  #L-shell of the simulation 
Bw_sim=10*10**(-9)

aeq0_deg=39.3   #initial equatorial pitch angle
aeq0=np.deg2rad(aeq0_deg) #convert pitch angle to rad

Ekev0=2000 #initial energy keV
lamda0_deg=0 # starting electron latitude
lamda0=np.deg2rad(lamda0_deg) #convert latitude to rad

f_wave=1.84 # wave frequency in Hz
w_wave=2*np.pi*f_wave #wave angular frequency

# eta0_deg=np.arange(0,361,30)#initial phases of electrons
eta0_deg=np.linspace(0,360,24)
eta0=np.deg2rad(eta0_deg) #convert initial phases to rad


B0=32200*10**(-9)
slat=np.sin(lamda0)
clat=np.cos(lamda0)
slat_term = np.sqrt(1 + 3*slat*slat)
B_sim=(B0/(L_sim**3))*slat_term/(clat**6)
    
wce_sim=cyclotron(B_sim,const.qe,const.me)[0]
wcHe_sim=cyclotron(B_sim,const.qe,const.mHe)[0]
wcO_sim=cyclotron(B_sim,const.qe,const.mO)[0]
wcH_sim=cyclotron(B_sim,const.qe,const.mH)[0]



slateq=np.sin(lamda0)
clateq=np.cos(lamda0)
slat_termeq = np.sqrt(1 + 3*slateq*slateq)
Beq=(B0/(L_sim**3))*slat_termeq/(clateq**6)

wce_sim_eq=cyclotron(Beq,const.qe,const.me)[0]
wcHe_sim_eq=cyclotron(Beq,const.qe,const.mHe)[0]
wmega_sim=0.96*wcHe_sim_eq

wpe_sim_eq=15*wce_sim_eq

ne_simeq=(wpe_sim_eq*wpe_sim_eq*const.me*const.epsilon0)/(const.qe*const.qe)

ne_sim=densities_denton(ne_simeq,lamda0)
nH_sim=0.77*ne_sim
nHe_sim=0.2*ne_sim
nO_sim=0.03*ne_sim
wpe_sim=omega_plasma(ne_sim,const.qe,const.mH)[0]
wpH_sim=omega_plasma(nH_sim,const.qe,const.mH)[0]
wpHe_sim=omega_plasma(nHe_sim,const.qe,const.mHe)[0]
wpO_sim=omega_plasma(nO_sim,const.qe,const.mO)[0]

alpha0=aeq2alpha(L_sim,lamda0,aeq0)
musq0=ref_index_parallel_EMIC(w_wave,wpe_sim,wce_sim,wpH_sim,wcH_sim,wpO_sim,wcO_sim,wpHe_sim,wcHe_sim)
kappa0=np.sqrt(musq0)*w_wave/const.c_light
upar0,uper0,ppar0,pper0,gamma0=momentums(Ekev0,alpha0,const.me)    


dwc_ds0=dwc_ds(wce_sim,lamda0,L_sim)
dk_dt=0

H0=nonlinear_H(pper0,ppar0,kappa0,gamma0,const.me,wce_sim,dk_dt,dwc_ds0,0)
thet0,wtsq0=nonlinear_theta(pper0,ppar0,Bw_sim,kappa0,gamma0,const.me,const.qe,wce_sim,wmega_sim)
S0=nonlinear_S(H0,wtsq0)

t=0.07 #simulation duration (s)
h=0.00001  #simulation stepsize
Nsteps=int(t/h) #number of simulation steps



### 2. Allocate outputs

In [ ]:
pper=np.zeros((len(eta0),Nsteps+1))
ppar=np.zeros((len(eta0),Nsteps+1))
eta=np.zeros((len(eta0),Nsteps+1))
lamda=np.zeros((len(eta0),Nsteps+1))
time=np.zeros((len(eta0),Nsteps+1))
uper=np.zeros((len(eta0),Nsteps+1))
upar=np.zeros((len(eta0),Nsteps+1))
zeta=np.zeros((len(eta0),Nsteps+1))
alpha=np.zeros((len(eta0),Nsteps+1))
aeq=np.zeros((len(eta0),Nsteps+1))
vresz_o=np.zeros((len(eta0),Nsteps+1))
Eres_o=np.zeros((len(eta0),Nsteps+1))
mu_adiabatic=np.zeros((len(eta0),Nsteps+1))
deta_dt=np.zeros((len(eta0),Nsteps+1))
kappa_out=np.zeros((len(eta0),Nsteps+1))
kx_out=np.zeros((len(eta0),Nsteps+1))
kz_out=np.zeros((len(eta0),Nsteps+1))
gamma=np.zeros((len(eta0),Nsteps+1))
E_kin=np.zeros((len(eta0),Nsteps+1))
ref_sq=np.zeros((len(eta0),Nsteps+1))
u_res_out=np.zeros((len(eta0),Nsteps+1))
H_out=np.zeros((len(eta0),Nsteps+1))
S_out=np.zeros((len(eta0),Nsteps+1))
detadt_out=np.zeros((len(eta0),Nsteps+1))
wtrsq_out=np.zeros((len(eta0),Nsteps+1))
thet_out=np.zeros((len(eta0),Nsteps+1))

### 3. Import WPIT routines

In [ ]:
from parallel_EMIC_mod import dzdt
from parallel_EMIC_mod import dppardt
from parallel_EMIC_mod import dpperdt
from parallel_EMIC_mod import detadt
from parallel_EMIC_mod import dlamdadt
from parallel_EMIC_mod import dalphadt
from parallel_EMIC_mod import daeqdt
from parallel_EMIC_mod import dgammadt
from parallel_EMIC_mod import dEkdt


In [ ]:
detasts_0=[]
for k in range(0,len(eta0)):
    eta0tmp=detadt(ppar0,pper0,eta0[k],Bw_sim,wmega_sim,kappa0,wce_sim,gamma0,const.qe,const.me)
    detasts_0.append(eta0tmp)

In [ ]:
for k in range(0,len(eta0)):
    i=0
    time[k,0]=0
    pper[k,0]=pper0
    ppar[k,0]=ppar0
    eta[k,0]=eta0[k]
    lamda[k,0]=lamda0
    zeta[k,0]=0
    upar[k,0]=upar0
    uper[k,0]=uper0
    alpha[k,0]=alpha0
    aeq[k,0]=aeq0
    gamma[k,0]=gamma0
    E_kin[k,0]=1.602176487E-16*Ekev0
    ref_sq[k,0]=musq0
    kappa_out[k,0]=kappa0
    H_out[k,0]=H0
    S_out[k,0]=S0
    detadt_out[k,0]=detasts_0[k]
    wtrsq_out[k,0]=wtsq0
    thet_out[k,0]=thet0
    print(np.rad2deg(eta[k,0]))
##############################################
    while i<Nsteps:
##############################################################################################################        
        #First step of Runge-Kutta
        if lamda[k,i]<np.deg2rad(28) and lamda[k,i]>-np.deg2rad(28):
            Bw_sim=Bw_sim
        else:
            Bw_sim=0

        B0=32200*10**(-9)
        slat=np.sin(lamda[k,i])
        clat=np.cos(lamda[k,i])
        slat_term = np.sqrt(1 + 3*slat*slat)
        Bmagtmp=(B0/(L_sim**3))*slat_term/(clat**6)

        n_etmp=densities_denton(ne_simeq,lamda[k,i])
        n_Htmp=0.77*n_etmp
        n_Hetmp=0.2*n_etmp
        n_Otmp=0.03*n_etmp

        wcetmp=cyclotron(Bmagtmp,const.qe,const.me)[0]
        wcHtmp=cyclotron(Bmagtmp,const.qe,const.mH)[0]
        wcHetmp=cyclotron(Bmagtmp,const.qe,const.mHe)[0]
        wcOtmp=cyclotron(Bmagtmp,const.qe,const.mO)[0]
        
        wpetmp=omega_plasma(n_etmp,const.qe,const.me)[0]
        wpHtmp=omega_plasma(n_Htmp,const.qe,const.mH)[0]
        wpHetmp=omega_plasma(n_Hetmp,const.qe,const.mHe)[0]
        wpOtmp=omega_plasma(n_Otmp,const.qe,const.mO)[0]

        musqtmp=ref_index_parallel_EMIC(w_wave,wpetmp,wcetmp,wpHtmp,wcHtmp,wpOtmp,wcOtmp,wpHetmp,wcHetmp)
        kappatmp=np.sqrt(musqtmp)*w_wave/const.c_light
        

        dwh_dstmp=dwc_ds(wcetmp,lamda[k,i],L_sim)
        
        p_mag=np.sqrt(ppar[k,i]*ppar[k,i]+pper[k,i]*pper[k,i])
        
        gammatmp=gamma[k,i]

        k1=dzdt(ppar[k,i],gammatmp,const.me)
        l1=dppardt(pper[k,i],eta[k,i],wcetmp,Bw_sim,gammatmp,dwh_dstmp,const.qe,const.me)
        m1=dpperdt(ppar[k,i],pper[k,i],eta[k,i],Bw_sim,gammatmp,wmega_sim,kappatmp,wcetmp,dwh_dstmp,const.qe,const.me)
        n1=detadt(ppar[k,i],pper[k,i],eta[k,i],Bw_sim,w_wave,kappatmp,wcetmp,gammatmp,const.qe,const.me)
        o1=dlamdadt(ppar[k,i],lamda[k,i],gammatmp,const.me,L_sim)
        p1=dalphadt(ppar[k,i],pper[k,i],eta[k,i],gammatmp,wcetmp,dwh_dstmp,w_wave,kappatmp,Bw_sim,const.me,const.qe)
        q1=daeqdt(ppar[k,i],pper[k,i],eta[k,i],gammatmp,w_wave,kappatmp,Bw_sim,aeq[k,i],alpha[k,i],const.me,const.qe)
        r1=dgammadt(pper[k,i],eta[k,i],gammatmp,Bw_sim,kappatmp,w_wave,const.qe,const.me)
        s1=dEkdt(pper[k,i],eta[k,i],gammatmp,Bw_sim,kappatmp,w_wave,const.qe,const.me)
        

        #Second step of Runge-Kutta
        if (lamda[k,i]+0.5*h*o1)<np.deg2rad(28) and (lamda[k,i]+0.5*h*o1)>-np.deg2rad(28):
            Bw_sim=Bw_sim
        else:
            Bw_sim=0
            
        B0=32200*10**(-9)
        slat=np.sin(lamda[k,i]+0.5*h*o1)
        clat=np.cos(lamda[k,i]+0.5*h*o1)
        slat_term = np.sqrt(1 + 3*slat*slat)
        Bmagtmp=(B0/(L_sim**3))*slat_term/(clat**6)
        
        n_etmp=densities_denton(ne_simeq,lamda[k,i]+0.5*h*o1)
        n_Htmp=0.77*n_etmp
        n_Hetmp=0.2*n_etmp
        n_Otmp=0.03*n_etmp

        wcetmp=cyclotron(Bmagtmp,const.qe,const.me)[0]
        wcHtmp=cyclotron(Bmagtmp,const.qe,const.mH)[0]
        wcHetmp=cyclotron(Bmagtmp,const.qe,const.mHe)[0]
        wcOtmp=cyclotron(Bmagtmp,const.qe,const.mO)[0]
        
        wpetmp=omega_plasma(n_etmp,const.qe,const.me)[0]
        wpHtmp=omega_plasma(n_Htmp,const.qe,const.mH)[0]
        wpHetmp=omega_plasma(n_Hetmp,const.qe,const.mHe)[0]
        wpOtmp=omega_plasma(n_Otmp,const.qe,const.mO)[0]

        musqtmp=ref_index_parallel_EMIC(w_wave,wpetmp,wcetmp,wpHtmp,wcHtmp,wpOtmp,wcOtmp,wpHetmp,wcHetmp)
        kappatmp=np.sqrt(musqtmp)*w_wave/const.c_light
        

        dwh_dstmp=dwc_ds(wcetmp,lamda[k,i]+0.5*h*o1,L_sim)
        
        p_mag=np.sqrt((ppar[k,i]+0.5*h*l1)*(ppar[k,i]+0.5*h*l1)+(pper[k,i]+0.5*h*m1)*(pper[k,i]+0.5*h*m1))
        
        gammatmp=gamma[k,i]+0.5*h*r1

        k2=dzdt(ppar[k,i]+0.5*h*l1,gammatmp,const.me)
        l2=dppardt(pper[k,i]+0.5*h*m1,eta[k,i]+0.5*h*n1,wcetmp,Bw_sim,gammatmp,dwh_dstmp,const.qe,const.me)
        m2=dpperdt(ppar[k,i]+0.5*h*l1,pper[k,i]+0.5*h*m1,eta[k,i]+0.5*h*n1,Bw_sim,gammatmp,wmega_sim,kappatmp,wcetmp,dwh_dstmp,const.qe,const.me)
        n2=detadt(ppar[k,i]+0.5*h*l1,pper[k,i]+0.5*h*m1,eta[k,i]+0.5*h*n1,Bw_sim,w_wave,kappatmp,wcetmp,gammatmp,const.qe,const.me)
        o2=dlamdadt(ppar[k,i]+0.5*h*l1,lamda[k,i]+0.5*h*o1,gammatmp,const.me,L_sim)
        p2=dalphadt(ppar[k,i]+0.5*h*l1,pper[k,i]+0.5*h*m1,eta[k,i]+0.5*h*n1,gammatmp,wcetmp,dwh_dstmp,w_wave,kappatmp,Bw_sim,const.me,const.qe)
        q2=daeqdt(ppar[k,i]+0.5*h*l1,pper[k,i]+0.5*h*m1,eta[k,i]+0.5*h*n1,gammatmp,w_wave,kappatmp,Bw_sim,aeq[k,i]+0.5*h*q1,alpha[k,i]+0.5*h*p1,const.me,const.qe)
        r2=dgammadt(pper[k,i]+0.5*h*m1,eta[k,i]+0.5*h*n1,gammatmp,Bw_sim,kappatmp,w_wave,const.qe,const.me)
        s2=dEkdt(pper[k,i]+0.5*h*m1,eta[k,i]+0.5*h*n1,gammatmp,Bw_sim,kappatmp,w_wave,const.qe,const.me)
        
#             ##############################################################################################################        
        #Third step of Runge-Kutta
        if (lamda[k,i]+0.5*h*o2)<np.deg2rad(28) and (lamda[k,i]+0.5*h*o2)>-np.deg2rad(28):
            Bw_sim=Bw_sim
        else:
            Bw_sim=0
            
        B0=32200*10**(-9)
        slat=np.sin(lamda[k,i]+0.5*h*o2)
        clat=np.cos(lamda[k,i]+0.5*h*o2)
        slat_term = np.sqrt(1 + 3*slat*slat)
        Bmagtmp=(B0/(L_sim**3))*slat_term/(clat**6)
        
        n_etmp=densities_denton(ne_simeq,lamda[k,i]+0.5*h*o2)
        n_Htmp=0.77*n_etmp
        n_Hetmp=0.2*n_etmp
        n_Otmp=0.03*n_etmp
        
        wcetmp=cyclotron(Bmagtmp,const.qe,const.me)[0]
        wcHtmp=cyclotron(Bmagtmp,const.qe,const.mH)[0]
        wcHetmp=cyclotron(Bmagtmp,const.qe,const.mHe)[0]
        wcOtmp=cyclotron(Bmagtmp,const.qe,const.mO)[0]
        
        wpetmp=omega_plasma(n_etmp,const.qe,const.me)[0]
        wpHtmp=omega_plasma(n_Htmp,const.qe,const.mH)[0]
        wpHetmp=omega_plasma(n_Hetmp,const.qe,const.mHe)[0]
        wpOtmp=omega_plasma(n_Otmp,const.qe,const.mO)[0]

        musqtmp=ref_index_parallel_EMIC(w_wave,wpetmp,wcetmp,wpHtmp,wcHtmp,wpOtmp,wcOtmp,wpHetmp,wcHetmp)
        kappatmp=np.sqrt(musqtmp)*w_wave/const.c_light
        

        dwh_dstmp=dwc_ds(wcetmp,lamda[k,i]+0.5*h*o2,L_sim)
        
        p_mag=np.sqrt((ppar[k,i]+0.5*h*l2)*(ppar[k,i]+0.5*h*l2)+(pper[k,i]+0.5*h*m2)*(pper[k,i]+0.5*h*m2))
        
        gammatmp=gamma[k,i]+0.5*h*r2

        k3=dzdt(ppar[k,i]+0.5*h*l2,gammatmp,const.me)
        l3=dppardt(pper[k,i]+0.5*h*m2,eta[k,i]+0.5*h*n2,wcetmp,Bw_sim,gammatmp,dwh_dstmp,const.qe,const.me)
        m3=dpperdt(ppar[k,i]+0.5*h*l2,pper[k,i]+0.5*h*m2,eta[k,i]+0.5*h*n2,Bw_sim,gammatmp,wmega_sim,kappatmp,wcetmp,dwh_dstmp,const.qe,const.me)
        n3=detadt(ppar[k,i]+0.5*h*l2,pper[k,i]+0.5*h*m2,eta[k,i]+0.5*h*n2,Bw_sim,w_wave,kappatmp,wcetmp,gammatmp,const.qe,const.me)
        o3=dlamdadt(ppar[k,i]+0.5*h*l2,lamda[k,i]+0.5*h*o2,gammatmp,const.me,L_sim)
        p3=dalphadt(ppar[k,i]+0.5*h*l2,pper[k,i]+0.5*h*m2,eta[k,i]+0.5*h*n2,gammatmp,wcetmp,dwh_dstmp,w_wave,kappatmp,Bw_sim,const.me,const.qe)
        q3=daeqdt(ppar[k,i]+0.5*h*l2,pper[k,i]+0.5*h*m2,eta[k,i]+0.5*h*n2,gammatmp,w_wave,kappatmp,Bw_sim,aeq[k,i]+0.5*h*q2,alpha[k,i]+0.5*h*p2,const.me,const.qe)
        r3=dgammadt(pper[k,i]+0.5*h*m2,eta[k,i]+0.5*h*n2,gammatmp,Bw_sim,kappatmp,w_wave,const.qe,const.me)
        s3=dEkdt(pper[k,i]+0.5*h*m2,eta[k,i]+0.5*h*n2,gammatmp,Bw_sim,kappatmp,w_wave,const.qe,const.me)
               
#             ##############################################################################################################        
        #Fourth step of Runge-Kutta
        if (lamda[k,i]+h*o3)<np.deg2rad(28) and (lamda[k,i]+h*o3)>-np.deg2rad(28):
            Bw_sim=Bw_sim
        else:
            Bw_sim=0

        B0=32200*10**(-9)
        slat=np.sin(lamda[k,i]+h*o3)
        clat=np.cos(lamda[k,i]+h*o3)
        slat_term = np.sqrt(1 + 3*slat*slat)
        Bmagtmp=(B0/(L_sim**3))*slat_term/(clat**6)
        
        n_etmp=densities_denton(ne_simeq,lamda[k,i]+h*o3)
        n_Htmp=0.77*n_etmp
        n_Hetmp=0.2*n_etmp
        n_Otmp=0.03*n_etmp

        wcetmp=cyclotron(Bmagtmp,const.qe,const.me)[0]
        wcHtmp=cyclotron(Bmagtmp,const.qe,const.mH)[0]
        wcHetmp=cyclotron(Bmagtmp,const.qe,const.mHe)[0]
        wcOtmp=cyclotron(Bmagtmp,const.qe,const.mO)[0]
        
        wpetmp=omega_plasma(n_etmp,const.qe,const.me)[0]
        wpHtmp=omega_plasma(n_Htmp,const.qe,const.mH)[0]
        wpHetmp=omega_plasma(n_Hetmp,const.qe,const.mHe)[0]
        wpOtmp=omega_plasma(n_Otmp,const.qe,const.mO)[0]

        musqtmp=ref_index_parallel_EMIC(w_wave,wpetmp,wcetmp,wpHtmp,wcHtmp,wpOtmp,wcOtmp,wpHetmp,wcHetmp)
        kappatmp=np.sqrt(musqtmp)*w_wave/const.c_light
        

        dwh_dstmp=dwc_ds(wcetmp,lamda[k,i]+h*o3,L_sim)
        
        p_mag=np.sqrt((ppar[k,i]+h*l3)*(ppar[k,i]+h*l3)+(pper[k,i]+h*m3)*(pper[k,i]+h*m3))
        
        gammatmp=gamma[k,i]+h*r3

        k4=dzdt(ppar[k,i]+h*l3,gammatmp,const.me)
        l4=dppardt(pper[k,i]+h*m3,eta[k,i]+h*n3,wcetmp,Bw_sim,gammatmp,dwh_dstmp,const.qe,const.me)
        m4=dpperdt(ppar[k,i]+h*l3,pper[k,i]+h*m3,eta[k,i]+h*n3,Bw_sim,gammatmp,wmega_sim,kappatmp,wcetmp,dwh_dstmp,const.qe,const.me)
        n4=detadt(ppar[k,i]+h*l3,pper[k,i]+h*m3,eta[k,i]+h*n3,Bw_sim,w_wave,kappatmp,wcetmp,gammatmp,const.qe,const.me)
        o4=dlamdadt(ppar[k,i]+h*l3,lamda[k,i]+h*o3,gammatmp,const.me,L_sim)
        p4=dalphadt(ppar[k,i]+h*l3,pper[k,i]+h*m3,eta[k,i]+h*n3,gammatmp,wcetmp,dwh_dstmp,w_wave,kappatmp,Bw_sim,const.me,const.qe)
        q4=daeqdt(ppar[k,i]+h*l3,pper[k,i]+h*m3,eta[k,i]+h*n3,gammatmp,w_wave,kappatmp,Bw_sim,aeq[k,i]+h*q3,alpha[k,i]+h*p3,const.me,const.qe)
        r4=dgammadt(pper[k,i]+h*m3,eta[k,i]+h*n3,gammatmp,Bw_sim,kappatmp,w_wave,const.qe,const.me)
        s4=dEkdt(pper[k,i]+h*m3,eta[k,i]+h*n3,gammatmp,Bw_sim,kappatmp,w_wave,const.qe,const.me)
        
        zeta[k,i+1]=zeta[k,i]+(h/6)*(k1+2*k2+2*k3+k4)
        ppar[k,i+1]=ppar[k,i]+(h/6)*(l1+2*l2+2*l3+l4)
        pper[k,i+1]=pper[k,i]+(h/6)*(m1+2*m2+2*m3+m4)
        eta[k,i+1]=(eta[k,i]+(h/6)*(n1+2*n2+2*n3+n4))
        lamda[k,i+1]=lamda[k,i]+(h/6)*(o1+2*o2+2*o3+o4)
        alpha[k,i+1]=alpha[k,i]+(h/6)*(p1+2*p2+2*p3+p4)
        aeq[k,i+1]=aeq[k,i]+(h/6)*(q1+2*q2+2*q3+q4)
        gamma[k,i+1]=gamma[k,i]+(h/6)*(r1+2*r2+2*r3+r4)
        E_kin[k,i+1]=E_kin[k,i]+(h/6)*(s1+2*s2+2*s3+s4)
        deta_dt[k,i+1]=(1/6)*(n1+2*n2+2*n3+n4)
        
        B0=32200*10**(-9)
        slat=np.sin(lamda[k,i+1])
        clat=np.cos(lamda[k,i+1])
        slat_term = np.sqrt(1 + 3*slat*slat)
        Bmagtmp=(B0/(L_sim**3))*slat_term/(clat**6)
        
        n_etmp=densities_denton(ne_simeq,lamda[k,i+1])
        n_Htmp=0.77*n_etmp
        n_Hetmp=0.2*n_etmp
        n_Otmp=0.03*n_etmp

        wcetmp=cyclotron(Bmagtmp,const.qe,const.me)[0]
        wcHtmp=cyclotron(Bmagtmp,const.qe,const.mH)[0]
        wcHetmp=cyclotron(Bmagtmp,const.qe,const.mHe)[0]
        wcOtmp=cyclotron(Bmagtmp,const.qe,const.mO)[0]
        
        wpetmp=omega_plasma(n_etmp,const.qe,const.me)[0]
        wpHtmp=omega_plasma(n_Htmp,const.qe,const.mH)[0]
        wpHetmp=omega_plasma(n_Hetmp,const.qe,const.mHe)[0]
        wpOtmp=omega_plasma(n_Otmp,const.qe,const.mO)[0]

        musqtmp=ref_index_parallel_EMIC(w_wave,wpetmp,wcetmp,wpHtmp,wcHtmp,wpOtmp,wcOtmp,wpHetmp,wcHetmp)
        kappatmp=np.sqrt(musqtmp)*w_wave/const.c_light
        kappa_out[k,i+1]=kappatmp        

        dwh_dstmp=dwc_ds(wcetmp,lamda[k,i+1],L_sim)
        
        ref_sq[k,i+1]=musqtmp

        upar[k,i+1]=ppar[k,i+1]/(const.me*gammatmp)
        uper[k,i+1]=pper[k,i+1]/(const.me*gammatmp)
        u_mag=np.sqrt(upar[k,i+1]**2+uper[k,i+1]**2)
        pmagnitude=np.sqrt(ppar[k,i+1]**2+pper[k,i+1]**2)


        
        dkpardt_run=(kappa_out[k,i]-kappa_out[k,i-1])/h
        #####################################
        Htmp=nonlinear_H(pper[k,i+1],ppar[k,i+1],kappatmp,gammatmp,const.me,wcetmp,dkpardt_run,dwh_dstmp,0)
        thetatmp,wtsqtmp=nonlinear_theta(pper[k,i+1],ppar[k,i+1],Bw_sim,kappatmp,gammatmp,const.me,const.qe,wcetmp,wmega_sim)
        Stmp=nonlinear_S(Htmp,wtsqtmp)
        H_out[k,i+1]=Htmp
        S_out[k,i+1]=Stmp
        wtrsq_out[k,i+1]=wtsqtmp
        thet_out[k,i+1]=thetatmp


        i=i+1
        time[k,i]=time[k,i-1]+h
        print('wt2',wtsqtmp)
        print('moms',pper[k,i],ppar[k,i])
        print('B k g',Bw_sim,kappatmp,gammatmp)
        print('ws',wcetmp,wmega_sim)


In [ ]:
thetatmp,wtsqtmp=nonlinear_theta(1.1434250774048123e-21,6.474984539869913e-22,Bw_sim,1.116347905350279e-05,4.914432421768577,const.me,const.qe,240745.80033774258,wmega_sim)

print(wtsqtmp)




In [ ]:
from matplotlib import cm

# for r in range(0,len(eta0)):
#     for i in range(0,len(aeq)-1):
#         print(r,i,np.rad2deg(lamda[r,i]),np.rad2deg(aeq[r,i]))

# for i in range(0,len(lamda[1,:])):
#     print(np.rad2deg(aeq[1,i]))
# from matplotlib import cm
# mask=np.isnan(aeq)

aeq = np.ma.array(aeq, mask=np.isnan(aeq))
print(np.max(aeq[1,-1]))
# print(np.shape(aeq[1,:]))
# dalpha=[]
# for r in range(0,len(eta0)):
#     ls=aeq[r,:].shape[1] - (~np.isnan(aeq[r,:]))[:, ::-1].argmax(1) - 1
#     print(ls)
#     ls=(~np.isnan(aeq[r,:])).cumsum(2).argmax(1)
#     dalphaf=np.rad2deg(aeq[r,ls])-aeq0_deg
#     print(eta0[r],np.rad2deg(aeq[r,-1]))
#     dalpha.append(dalphaf)
    

dalpha=[]
for r in range(0,len(eta0)):
    ls=np.max(np.nonzero(aeq[r,:]))
    dalphaf=np.rad2deg(aeq[r,ls])-aeq0_deg
    dalpha.append(dalphaf)
# # print(dalpha)
norm = cm.colors.Normalize(vmin=eta0.min(), vmax=eta0.max())
cmap = cm.ScalarMappable(norm=norm, cmap=cm.jet)
cmap.set_array([])
#################
fig, ax = plt.subplots(figsize=(9,6))
s=5


for r in range(0,len(eta0)):
    ax.plot(np.rad2deg(lamda[r,:-1]),np.rad2deg(aeq[r,:-1]),c=cmap.to_rgba(eta0[r]))
ax.grid(alpha=.3)
ax.set_xlim(0,30)
ax.set_xlabel('Latitude (deg)')
ax.set_ylabel(r'$a_{eq}$ (deg)')

# ax.set_ylim(40,65)
ticks=np.arange(0,2*np.pi,1)
cbar=fig.colorbar(cmap, ticks=ticks)
cbar.set_label('gyro-phase (rad)', rotation=270,labelpad=15)
#     ax.axvline(x=-5,color="black", linestyle="--")
plt.show()

In [ ]:
fonts=15
wtr=np.sqrt(wtrsq_out)

nplot=deta_dt/wtr
# print(wtr)


fig, ax = plt.subplots(figsize=(10,6))
s=5
last=1

for r in range(0,len(eta0)):
    ax.plot(eta[r,2:-last]/np.pi,nplot[r,2:-last],c=cmap.to_rgba(eta0[r]))

ax.grid(alpha=.3)
ax.set_xlim(-1,18)
ax.set_xlabel(r'$\eta/\pi$',fontsize=fonts)
ax.set_ylabel(r'$\nu/\omega_t$',fontsize=fonts)

ax.set_ylim(-5,5)

cbar=fig.colorbar(cmap, ticks=[-np.pi,-np.pi/2,0,np.pi/2,np.pi])
cbar.ax.set_yticklabels([r'$-\pi$',r'$-\pi/2$', '0',r'$\pi/2$', r'$\pi$'])  # horizontal colorbar
cbar.set_label(r'$\eta_0$ (rad)', rotation=270,labelpad=15,fontsize=fonts)
# ax.axvline(x=-5,color="black", linestyle="--")
plt.show()


In [ ]:
######################
fig, ax = plt.subplots(figsize=(9,6))
s=5


for r in range(0,len(eta0)):
    ax.plot(time[r,:-1],np.rad2deg(aeq[r,:-1]),c=cmap.to_rgba(eta0[r]), linewidth=1)
ax.grid(alpha=.3)
# ax.set_xlim(0,30)
ax.set_ylabel(r'$a_{eq}$ (deg)')
ax.set_xlabel(r'$time$ (s)')

# ax.set_ylim(40,65)
ticks=np.arange(0,2*np.pi,1)
cbar=fig.colorbar(cmap, ticks=ticks)
cbar.set_label('gyro-phase (rad)', rotation=270,labelpad=15)
#     ax.axvline(x=-5,color="black", linestyle="--")
plt.show()
#####################################################

In [ ]:
#####################################################
fig, ax = plt.subplots(figsize=(9,6))
s=5

colors=eta0[:]
# for r in range(0,len(eta0)):
#     ax.plot(np.rad2deg(lamda[r,:-1]),np.rad2deg(aeq[r,:-1]),c=cmap.to_rgba(eta0[r]))
ax.grid(alpha=.3)
ax.plot(np.rad2deg(eta0),dalpha)
ax.scatter(np.rad2deg(eta0),dalpha,marker='o',facecolors='none', edgecolors=cmap.to_rgba(eta0[:]),s=100)
ax.set_xlim(0,360)
ax.set_xlabel(r'$\eta_0$ (deg)')
ax.set_ylabel(r'$\Delta a_{eq}$ (deg)')

# ax.set_ylim(-10,10)
ticks=np.arange(0,2*np.pi,1)
# cbar=fig.colorbar(cmap, ticks=ticks)
# cbar.set_label('gyro-phase (rad)', rotation=270,labelpad=15)
#     ax.axvline(x=-5,color="black", linestyle="--")
plt.show()
#################

In [ ]:

fig, ax = plt.subplots(figsize=(9,6))
s=5


for r in range(0,len(eta0)):
    ax.plot(np.rad2deg(lamda[r,:-1]),np.rad2deg(deta_dt[r,:-1]),c=cmap.to_rgba(eta0[r]))
ax.grid(alpha=.3)
ax.set_xlim(0,10)
ax.set_xlabel('Latitude (deg)')
ax.set_ylabel(r'$d\eta/dt$ ')

ax.set_ylim(-1*10**6,1*10**6)
ticks=np.arange(0,2*np.pi,1)
cbar=fig.colorbar(cmap, ticks=ticks)
cbar.set_label('gyro-phase (rad)', rotation=270,labelpad=15)
#     ax.axvline(x=-5,color="black", linestyle="--")
plt.show()
#####################################################

In [ ]:
#################
fig, ax = plt.subplots(figsize=(9,6))
s=5


for r in range(0,len(eta0)):
    ax.plot(np.rad2deg(lamda[r,:-1]),ppar[r,:-1],c=cmap.to_rgba(eta0[r]))
ax.grid(alpha=.3)
ax.set_xlim(0,30)
ax.set_xlabel('Latitude (deg)')
ax.set_ylabel(r'$p_{\parallel}$')

# ax.set_ylim(-2,2)
ticks=np.arange(0,2*np.pi,1)
cbar=fig.colorbar(cmap, ticks=ticks)
cbar.set_label('gyro-phase (rad)', rotation=270,labelpad=15)
#     ax.axvline(x=-5,color="black", linestyle="--")
plt.show()
#####################################################

In [ ]:

#################
fig, ax = plt.subplots(figsize=(9,6))
s=5


for r in range(0,len(eta0)):
    ax.plot(np.rad2deg(lamda[r,:-1]),pper[r,:-1],c=cmap.to_rgba(eta0[r]))
ax.grid(alpha=.3)
ax.set_xlim(0,30)
ax.set_xlabel('Latitude (deg)')
ax.set_ylabel(r'$p_{\perp}$')

# ax.set_ylim(-2,2)
ticks=np.arange(0,2*np.pi,1)
cbar=fig.colorbar(cmap, ticks=ticks)
cbar.set_label('gyro-phase (rad)', rotation=270,labelpad=15)
#     ax.axvline(x=-5,color="black", linestyle="--")
plt.show()

In [ ]:
#################
fig, ax = plt.subplots(figsize=(9,6))
s=5


for r in range(0,len(eta0)):
    ax.plot(np.rad2deg(lamda[r,:-1]),ref_sq[r,:-1],c=cmap.to_rgba(eta0[r]))
ax.grid(alpha=.3)
ax.set_xlim(0,30)
ax.set_xlabel('Latitude (deg)')
ax.set_ylabel('Refractive index squared')

# ax.set_ylim(-2,2)
ticks=np.arange(0,2*np.pi,1)
cbar=fig.colorbar(cmap, ticks=ticks)
cbar.set_label('gyro-phase (rad)', rotation=270,labelpad=15)
#     ax.axvline(x=-5,color="black", linestyle="--")
plt.show()
#####################################################

In [ ]:

#################
fig, ax = plt.subplots(figsize=(9,6))
s=5


for r in range(0,len(eta0)):
    ax.plot(np.rad2deg(lamda[r,:-1]),E_kin[r,:-1]*6.2415e15,c=cmap.to_rgba(eta0[r]))
ax.grid(alpha=.3)
ax.set_xlim(0,30)
ax.set_xlabel('Latitude (deg)')
ax.set_ylabel('E (MeV)')

# ax.set_ylim(-1,2.5)
ticks=np.arange(0,2*np.pi,1)
cbar=fig.colorbar(cmap, ticks=ticks)
cbar.set_label('gyro-phase (rad)', rotation=270,labelpad=15)
#     ax.axvline(x=-5,color="black", linestyle="--")
plt.show()
#####################################################


In [ ]:

##############
fig, ax = plt.subplots(figsize=(9,6))
s=5


for r in range(0,len(eta0)):
    ax.plot(np.rad2deg(lamda[r,:-1]),gamma[r,:-1],c=cmap.to_rgba(eta0[r]))
ax.grid(alpha=.3)
ax.set_xlim(0,30)
ax.set_xlabel('Latitude (deg)')
ax.set_ylabel(r'$\gamma$ (MeV)')

# ax.set_ylim(-1,2.5)
ticks=np.arange(0,2*np.pi,1)
cbar=fig.colorbar(cmap, ticks=ticks)
cbar.set_label('gyro-phase (rad)', rotation=270,labelpad=15)
#     ax.axvline(x=-5,color="black", linestyle="--")
plt.show()

